# ACF Installer

This notebook installs the ACF, using the pickle files generated from the released code. 

**NOTE:** this notebook should only be executed in the account where native app resides and will be deployed from.


## PREREQUISITE: Enable GitHub External Access Integration

- If your account ***does not*** have an External Access Integration for the GitHub API, execute the commands below:
```
CREATE OR REPLACE NETWORK RULE gh_network_rule
MODE = EGRESS
TYPE = HOST_PORT
VALUE_LIST = ('github.com', 'api.github.com');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION gh_access_integration
ALLOWED_NETWORK_RULES = (gh_network_rule)
ENABLED = true;
```
- Once the External Access Integration for the GitHub API as been created (or if one already exists), enable this notebook to use it, by following the steps here: https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-external-access#enable-external-access-integrations-eai.
    - **NOTE:** this step restarts the notebook.



## STEP 1: Initialization

In [1]:
import base64
import codecs
import io
import os
import pandas as pd
import pickle
import re
import requests
import sqlparse
import streamlit as st

session = get_active_session()

#tag session
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"acf","version":{{"major":1, "minor":7}},"attributes":{{"env":"acf","component":"installer","type":"notebook"}}'""").collect()

#get current_role
current_role = session.get_current_role().replace('"','')

st.success(f"Session initialized for role: {current_role} 🎉")

## STEP 2: Function definition

In [ ]:
def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)

def get_pickle(path):
    req = requests.get(path)
    if req.status_code == requests.codes.ok:
        req = req.json()
        content = base64.b64decode(req['content']).decode("utf-8")
        return content
    else:
        return 'Content was not found.'

def put_to_stage(session, stage, filename, type, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)

    if type.lower() == "file":
        f = open(local_file, "w", encoding='utf-8')
        f.write(content)
        f.close()

    if type.lower() == "image":
        with open(local_file, "wb") as f:
            img_data = content.encode()
            f.write(base64.b64decode(img_data))
            f.close()
        
    session.file.put(local_file, f"@{stage}", auto_compress=False, overwrite=True)
    return f"saved {type} {filename} in stage {stage}"

## STEP 3: Set ACF Account Parameters

In [ ]:
#get account org and locator
acf_acct_locator = session.get_current_account().replace('"','')

acf_app_code = st.text_input("Enter the ACF App Code 👇", help="The unique identifier for your native app (i.e. `ACME`). It is the same identifier specified when setting up the event account(s).")

## STEP 4: Get ACF pickle file and decode

In [ ]:
decoded_acf_obj = None

acf_pickle = get_pickle('https://api.github.com/repos/Snowflake-Labs/sfguide-application-control-framework/contents/pickles/02_acf.pickle')

if acf_pickle != 'Content was not found.':
    decoded_acf_obj = decode(acf_pickle)
    st.success(f"ACF Pickle Decoded 🎉")
else:
    print ('Content was not found.')

## STEP 5: Execute commands from decoded ACF pickle file

In [ ]:
verbose = st.selectbox("Verbose?", ("Select...", "Y", "N"), index=0,)

#acf setup scripts
acf_setup_list = decoded_acf_obj['acf_setup'].items()
for file_name, file_content in acf_setup_list:
    file_content = str(file_content.decode("utf-8"))
    #replace SnowSQL variables with values from Step 3 and comment out SnowSQL print/set commands, also comment out PUT commands
    repl = {"&{APP_CODE}": f"{acf_app_code}"
            ,"&APP_CODE": f"{acf_app_code}"
            ,"&{ACF_ACCOUNT_LOCATOR}": f"{acf_acct_locator}"
            ,"&ACF_ACCOUNT_LOCATOR": f"{acf_acct_locator}"
            ,"&&&&": "&&"
            ,"!print": "--!print"
            ,"!set": "--!set"
            ,"PUT 'file": "--PUT 'file"
          }

    repl = dict((re.escape(k), v) for k, v in repl.items()) 
    pattern = re.compile("|".join(repl.keys()))
    file_content = pattern.sub(lambda m: repl[re.escape(m.group(0))], file_content)

    #format file_content
    file_content = sqlparse.format(file_content, strip_comments=True).strip()

    #execute each sql statement
    statements = sqlparse.split(file_content)
    for stmt in statements:
        if verbose != "Select...":
            if verbose == "Y":
                st.code(f"""Statement executed: {stmt}
                """)
            session.sql(stmt).collect()

#dev environment templates
acf_dev_env_templates_list = decoded_acf_obj['acf_dev_env_templates'].items()
for file_name, file_content in acf_dev_env_templates_list:
    file_content = str(file_content.decode("utf-8"))

    if verbose != "Select...":
        #put file on stage
        stage = f"P_{acf_app_code}_SOURCE_DB_DEV.ARTIFACTS.ARTIFACTS/templates"
        put_to_stage(session, stage, file_name, "file", file_content)
        st.code(f"""File: {file_name} placed on stage: {stage}
                """)

#acf App Control Manager
acf_streamlit_main = decoded_acf_obj['acf_streamlit_main'].items()
for file_name, file_content in acf_streamlit_main:
    file_content = str(file_content.decode("utf-8"))

    if verbose != "Select...":
        #put file on stage
        stage = f"P_{acf_app_code}_ACF_DB.ACF_STREAMLIT.ACF_STREAMLIT"
        put_to_stage(session, stage, file_name, "file", file_content)
        st.code(f"""File: {file_name} placed on stage: {stage}
                """)

acf_streamlit_code_list = decoded_acf_obj['acf_streamlit_code'].items()
for file_name, file_content in acf_streamlit_code_list:
    file_content = str(file_content.decode("utf-8"))

    if verbose != "Select...":
        #put file on stage
        stage = f"P_{acf_app_code}_ACF_DB.ACF_STREAMLIT.ACF_STREAMLIT/acf"
        put_to_stage(session, stage, file_name, "file", file_content)
        st.code(f"""File: {file_name} placed on stage: {stage}
                """)

acf_streamlit_img_list = decoded_acf_obj['acf_streamlit_imgs'].items()
for file_name, file_content in acf_streamlit_img_list:
    file_content = str(file_content.decode("utf-8"))

    if verbose != "Select...":
        #put file on stage
        stage = f"P_{acf_app_code}_ACF_DB.ACF_STREAMLIT.ACF_STREAMLIT/img"
        put_to_stage(session, stage, file_name, "image", file_content)
        st.code(f"""File: {file_name} placed on stage: {stage}
                """)

    if file_name == list(acf_streamlit_img_list)[-1][0]:
        st.success(f"ACF Installed 🎉")